<a href="https://colab.research.google.com/github/khadidja2005/Fine_tuned_SQL_Generator/blob/main/SQL_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install transformers accelerate peft datasets
!pip install huggingface_hub
!pip install -U transformers bitsandbytes
!pip show transformers bitsandbytes
from huggingface_hub import login
from google.colab import userdata
from transformers import BitsAndBytesConfig
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from peft import get_peft_model, LoraConfig
from transformers import Trainer, TrainingArguments

Name: transformers
Version: 4.46.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
---
Name: bitsandbytes
Version: 0.44.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [41]:
from google.colab import userdata
import os
from huggingface_hub import login

# Retrieve the token from an environment variable
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
login(token=huggingface_token, add_to_git_credential=True)

In [42]:
if torch.cuda.is_available():
    print("GPU is available")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")

GPU is available
Device name: Tesla T4


In [43]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name ,use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
import json
from datasets import Dataset

# Load data from JSON file
with open("/content/data.json", "r") as f:
    data = json.load(f)

# Convert to Hugging Face Dataset format
dataset = Dataset.from_list(data)

In [45]:
dataset

Dataset({
    features: ['prompt', 'response'],
    num_rows: 102
})

In [46]:
# Tokenize the dataset
def preprocess_function(examples):
    # Concatenate prompt and response for input-output pairs
    inputs = examples['prompt']
    outputs = examples['response']
    tokenizer.pad_token = tokenizer.eos_token
    model_inputs = tokenizer(inputs, padding = "max_length",max_length=128 , truncation=True, return_tensors="pt")
    model_outputs = tokenizer(outputs, padding="max_length",max_length=128, truncation=True, return_tensors="pt")

    # Labels are the same as the output tokens
    model_inputs['labels'] = model_outputs['input_ids']
    return model_inputs

# Preprocess data
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [47]:
tokenized_dataset

Dataset({
    features: ['prompt', 'response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 102
})

In [48]:
peft_config = LoraConfig(
    r=8,             # Number of attention heads to fine-tune
    lora_alpha=32,    # Alpha parameter for scaling
    target_modules=["q_proj", "v_proj"],  # Layers to fine-tune
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, peft_config)

In [49]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [50]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=4,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    remove_unused_columns=False

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


TrainOutput(global_step=24, training_loss=6.820037841796875, metrics={'train_runtime': 115.0478, 'train_samples_per_second': 3.546, 'train_steps_per_second': 0.209, 'total_flos': 2066155604803584.0, 'train_loss': 6.820037841796875, 'epoch': 3.6923076923076925})

In [51]:
prompt = "What is the revenue of the company with ID 123 ?"
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_length=100, num_return_sequences=1)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


What is the revenue of the company with ID 123 ?

```
SELECT revenue
FROM company
WHERE id = 123;
```

What is the name of the company with ID 123 ?

```
SELECT name
FROM company
WHERE id = 123;
```

What is the average revenue of all companies in the 'retail' industry ?

```



In [52]:
prompt = "What is the revenue of the company with ID 999 ?"
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_length=100, num_return_sequences=1)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is the revenue of the company with ID 999 ?

```
SELECT revenue
FROM company
WHERE id = 999;
```

Assuming the `company` table has columns `id` and `revenue`.

This query will return the revenue of the company with ID 999. If no company with ID 999 exists, it will return an error.

If you want to handle the


In [53]:
prompt = "Get profit trends by quarter for each organization ?"
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_length=100, num_return_sequences=1)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Get profit trends by quarter for each organization ?

You can get profit trends by quarter for each organization by using the following steps:

1. Query the `profit` and `quarter` fields from the `organizations` table.

```sql
SELECT organization_id, quarter, SUM(profit) as total_profit
FROM organizations
GROUP BY organization_id, quarter
ORDER BY organization_id, quarter;
```

2.


In [54]:
prompt = "Get average expenses spent by company , company id is 345 ?"
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_length=100, num_return_sequences=1)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Get average expenses spent by company , company id is 345 ?

```
SELECT AVG(expenses) FROM expenses WHERE company_id = 345;
```

This SQL query calculates the average expenses for a specific company with the id 345. The `AVG()` function is used to calculate the average, and the `WHERE` clause is used to filter the results to only include expenses for the company with
